In [ ]:
#imports and set up

from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from IPython.display import display
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import time
import sys
from process_pickle import process_pickle
from process_pickle_quiver import process_pickle_quiver
%matplotlib inline
from ipywidgets import interact, interactive, fixed #Sliders for image selection
import ipywidgets as widgets
mpl.rc('figure',  figsize=(4.77, 2.95))
mpl.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
mpl.rcParams['lines.linewidth'] = 0.6
mpl.rcParams['lines.color'] = 'r'
plt.rc('grid', linestyle="--", color='gray')

import numpy as np
import pandas as pd
from pandas import DataFrame, Series  

import pims
import trackpy as tp
import os

In [ ]:
# function that is called to configure how plots are to be drawn

def setup_plot():
    ax = plt.gca()
    SPINE_COLOR = 'gray'
    for spine in ['top', 'right']:
            ax.spines[spine].set_visible(False)

    for spine in ['left', 'bottom']:
        ax.spines[spine].set_color(SPINE_COLOR)
        ax.spines[spine].set_linewidth(0.5)

    ax.xaxis.set_ticks_position('bottom')
    ax.yaxis.set_ticks_position('left')

    for axis in [ax.xaxis, ax.yaxis]:
        axis.set_tick_params(direction='out', color=SPINE_COLOR)
    plt.grid()
    fig = plt.gcf()
    fig.set_size_inches(4.77, 2.95, forward=True)

Use the file picker to locate the .pkl containing the particle trajectories

or select a previously output .xlsx to skip the calculations

**If it doesn't appear you probably need to minimise your browser or alt tab around for it**

In [ ]:
set_custom_data_directory = True

In [ ]:
# select the .pkl files you wish to analyse. When you're done selecting cancel should
# allow you to continue to the next cell.

import tkinter as tk
from tkinter import filedialog

pkl_to_process = []
file_path = "1"
while len(file_path) != 0:
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename()
    if len(file_path) != 0:
        pkl_to_process.append(file_path)

In [ ]:
# confirm you've selected the correct files

pkl_to_process

In [ ]:
calculate = True

In [ ]:
# configure the calculations to be carried out on the data contained in the .pkl files
# if you don't enter "NEXT" at the end you can queue another set of calculations to be carried out on the 
# same .pkl, e.g with a frame lookback.
pkl_param = []

i = 0
mpp = 15 # microns per pixel
timestep = 30 # seconds between images

while i < len(pkl_to_process):
    print(pkl_to_process[i])
    print("Setting Parameters...")
    # uncomment the below lines to set mpp or timestep on a per .pkl basis
    #mpp = int(input("Microns per Pixel: "))
    #timestep = int(input("Time between frames in seconds: "))
    frame_lookback = int(input("How many frames to look back for velocity calculations? "))
    pkl_param.append([i,mpp,timestep,frame_lookback])
    move_next = input("Enter NEXT to move to next pkl, anything else to add another analysis run for the current particle")
    if move_next == "NEXT":
        i += 1
    

In [ ]:
pkl_param

In [ ]:
# for the sets of calculations you've requested, select folders to store the results for each set in.

data_dirs = []
data_dir = True
counter = 0
for i in pkl_param:
    root = tk.Tk()
    root.withdraw()
    data_dir = filedialog.askdirectory(title="Pick data dir for {} {}".format(pkl_to_process[i[0]], i[3]))
    data_dirs.append(data_dir)

In [ ]:
# if the folders you've requested the results stored in don't exist, make them:

for directory in data_dirs:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)

for c, param in enumerate(pkl_param):
    # the process_pickle_quiver function is contained within a separate file - process_pickle_quiver.py
    df = process_pickle_quiver(pkl_to_process[param[0]], data_dirs[c], param[1], param[2], param[3])

In [ ]:
# function called to draw the charts of the results

def produce_charts(df, general_directory):
    if not os.path.exists(general_directory + "/figures"):
        os.makedirs(general_directory + "/figures")
    no_bins = df.bin.max() + 1    
    av_disp_by_frame = df.groupby('frame').displacement.mean().sort_index()
    xs = av_disp_by_frame.index.values
    ys = av_disp_by_frame.values
    plt.figure()
    setup_plot()
    plt.plot(xs, ys)
    plt.ylabel("Displacement / px")
    plt.xlabel("Frame")
    plt.tight_layout()
    plt.savefig(general_directory + "/figures/Average_displacement.pdf")
    fig = plt.gcf()


    disp_vels = df.groupby('particle').velocity.mean()
    plt.figure()
    setup_plot()
    plt.xlabel("Velocity / ms$^-1$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.hist(disp_vels, bins=20)
    plt.savefig(general_directory + "/figures/Velocity Distribution by Displacement.pdf")


    y_vels = df.groupby('particle').y_velocity.mean()
    plt.figure()
    setup_plot()
    plt.xlabel("Velocity / ms$^-1$")
    plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
    plt.hist(y_vels, bins=20)
    plt.savefig(general_directory + "/figures/Velocity Distribution by Y Velocity.pdf")



    av_vel_by_frame = df.groupby('frame').velocity.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.tight_layout()
    plt.plot(xs, ys,)
    plt.savefig(general_directory + "/figures/Average_Velocity by Displacement.pdf")

    av_vel_by_frame = df.groupby('frame').y_velocity.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.plot(xs, ys)
    plt.savefig(general_directory + "/figures/Average_Velocity by Y Position.pdf")
    
    av_vel_by_frame = df.groupby('frame').x_velocity.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.tight_layout()
    plt.plot(xs, ys)
    plt.savefig(general_directory + "/figures/Average_Velocity by X Position.pdf")
    
    av_vel_by_frame = df.groupby('frame').hyp_vel.mean().sort_index()
    xs = av_vel_by_frame.index.values
    ys = av_vel_by_frame.values
    plt.figure()
    setup_plot()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.autoscale(enable=True, axis='both', tight=True)
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("Frame")
    plt.autoscale(True)
    plt.tight_layout()
    plt.plot(xs, ys)

    plt.savefig(general_directory + "/figures/Average_Velocity.pdf")

    spacings = []
    for x in df.frame.unique():
        w_df = df[df['frame']==x]
        spacings.append(tp.proximity(w_df)['proximity'].mean())
    plt.figure()
    setup_plot()
    plt.ylabel("Distance / px")
    plt.xlabel("Frame")
    plt.plot(spacings)
    plt.savefig(general_directory + "/figures/nearest_neighbour.pdf")

    bin_vels = df.groupby('bin').hyp_vel.mean()
    plt.figure()
    bin_vels_to_plot = []
    for c, v in enumerate(bin_vels):
        bin_vels_to_plot.append((c, v))

    err = df.groupby('bin').hyp_vel.std()
    x = [x[0] for x in bin_vels_to_plot]
    y = [x[1] for x in bin_vels_to_plot]
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    setup_plot()
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("x position (" + str(no_bins) +" equally spaced bins)")
    plt.tight_layout()
    plt.bar(x, y, yerr = err, error_kw=dict(elinewidth=2,ecolor='black'))

    plt.savefig(general_directory + "/figures/x_binned_velocity.pdf")
    
    bin_vels = df.groupby('bin').y_velocity.mean()
    plt.figure()
    bin_vels_to_plot = []
    for c, v in enumerate(bin_vels):
        bin_vels_to_plot.append((c, v))

    err = df.groupby('bin').y_velocity.std()
    x = [x[0] for x in bin_vels_to_plot]
    y = [x[1] for x in bin_vels_to_plot]
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    setup_plot()
    plt.ylabel("Velocity / ms$^-1$")
    plt.xlabel("x position (" + str(no_bins) +" equally spaced bins)")
    plt.tight_layout()
    plt.bar(x, y, yerr = err, error_kw=dict(elinewidth=2,ecolor='black'))

    plt.savefig(general_directory + "/figures/x_binned_y_velocity.pdf")
    
    

    def x_hist(frame):
        w_df = df[df.frame == frame].copy()
        bin_vels = w_df.groupby('bin').hyp_vel.mean()
        plt.figure()
        bin_vels_to_plot = []
        for c, v in enumerate(bin_vels):
            bin_vels_to_plot.append((c, v))
        x = [x[0] for x in bin_vels_to_plot]
        y = [x[1] for x in bin_vels_to_plot]
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        setup_plot()
        plt.ylabel("Velocity / ms$^-1$")
        plt.xlabel("x position (" + str(no_bins) +" equally spaced bins)")
        plt.xlim(0, 20)
        plt.bar(x, y)



    ### Set use_y_vel_only to true to plot only the y component of the particles' velocities

    from mpl_toolkits.mplot3d import Axes3D
    use_y_vel_only = True

    if use_y_vel_only:
        a = df.groupby([(df.frame // 50) * 50 , 'bin']).y_velocity.mean().fillna(0)
    else:
        a = df.groupby([(df.frame // 50) * 50 , 'bin']).hyp_vel.mean().fillna(0)

    xs = []
    ys = []
    zs = []
    xs = [x[1] for x in a.index.values]
    zs = [x[0] for x in a.index.values]

    for x in a:
        ys.append(x * 1*10**6)  



    colours_alt = []
    options = ['r','g', 'b']
    option_index = 0
    prev_a = 99
    for a in xs:
        if a <= prev_a:
            option_index += 1
            if (option_index)  >= len(options):
                option_index = 0
        colours_alt.append(options[option_index])
        prev_a = a  

    r = 0.99
    colours = []
    add = r / len(xs)
    for x in range(len(xs)):
        colours.append((r, 0, 1 -r))
        r -= add

    %matplotlib inline
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.bar(xs, ys, zs=zs, zdir = 'y', color = colours)
    ax.set_xlabel('x bin')
    ax.set_zlabel('Average velocity / um s-1')
    ax.set_ylabel('Frame')
    plt.show()
    plt.savefig(general_directory + "/figures/3d_bar_vel.pdf")

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(xs, zs, ys, color = colours)
    ax.set_xlabel('x bin')
    ax.set_zlabel('Average velocity / um s-1')
    ax.set_ylabel('Frame')
    plt.show() 

    ax = df.groupby('frame')['x'].count().plot()
    fig = ax.get_figure()
    fig.savefig(general_directory + "/figures/particles_in_frame.pdf")

for c, param in enumerate(pkl_param):
    # the process_pickle function is contained within a separate file - process_pickle.py
    df = process_pickle(pkl_to_process[param[0]], data_dirs[c], param[1], param[2], param[3])
    print('Producing plots...')
    print('\n')
    produce_charts(df, data_dirs[c])    
    print('\n')
    print('Finished')  

In [ ]:
from scipy import *
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from process_pickle_quiver import process_pickle_quiver
import pandas as pd
from scipy.interpolate import griddata
import timeit

def quiver_df(qv, general_directory):
    if not os.path.exists(general_directory + "/figures"):
        os.makedirs(general_directory + "/figures")
    
    #import ipdb; ipdb.set_trace() # debugging for quiver_plot function
    
    df = pd.read_csv(general_directory + '/data_quiver.csv')
    qv = pd.read_csv(general_directory + '/data_quiver.csv', usecols=[9,10,15,16,17,18,19])   
    print('\n')
    print('Raw data: ')
    print('\n')
    print(qv.head())
    particles = df.particle.unique() 
    
    qv['frameno'] = qv['frame']
    qv['velabs'] = qv.hyp_vel.abs()
    qv['hyp_vel'] = (qv['hyp_vel'])/(qv['velabs'].max())
    qv['nxvel'] = (qv['x_velocity'])/(qv['velabs'].max())
    qv['nyvel'] = (qv['y_velocity'])/(qv['velabs'].max())
    qv = qv.drop(columns=['velabs', 'x_velocity', 'y_velocity'])
    
    df['nyvel'] = qv['nyvel']
    df['nxvel'] = qv['nxvel']
    df['hyp_vel'] = qv['hyp_vel']
    step = int(input('How many frames per animation frame? '))
    qv.reset_index()
    qv.set_index(['particle', 'frame'], inplace=True)
    print('\n')
    print('Setting index by particle and frame number: ')
    print('\n')
    print(qv.head())
    qv = qv.dropna()
    print('\n')
    
    for c, particle in enumerate(particles):
        sys.stdout.write("\r Creating dataframes for particles: {:.2%}".format(c/len(particles)))
        sys.stdout.flush()
        p = df[df.particle == particle]
        
        frames = p.frame.unique()
        
        for i in p.index:
            
            if (i % step == 0):
                p.rolling(step).mean()
                x = p.loc[i].binx.item()
                y = p.loc[i].biny.item()
                nxvel = p.loc[i].nxvel.item()
                nyvel = p.loc[i].nyvel.item()
                hyp_vel = p.loc[i].hyp_vel.item()
                
                qv['binx'].loc[i] = x
                qv['biny'].loc[i] = y
                qv['nxvel'].loc[i] = nxvel
                qv['nxvel'].loc[i] = nyvel
                qv['hyp_vel'].loc[i] = hyp_vel          
                
            else:
                qv['binx'].loc[i] = np.nan
                qv['biny'].loc[i] = np.nan
                qv['nxvel'].loc[i] = np.nan
                qv['nyvel'].loc[i] = np.nan
                qv['hyp_vel'].loc[i] = np.nan  
                
            qv['frameno'].loc[i] = i
    
    qv.index = qv.index.droplevel(0)
    qv = qv.sort_index()
    qv = qv.reset_index()
    qv['frame'] = qv['frameno']
    qv = qv.drop(columns=['frameno'])
 
    qv = qv.dropna()
    print('\n')
    print('Dropping empty values: ')
    print('\n')
    print(qv.head())
    print('\n')
    

        
    frames = qv.frame.unique()
        
    for i in frames: 
        sys.stdout.write("\r Averaging across (x, y) co-ordinates: {:.2%}".format(i/len(frames)))
        sys.stdout.flush()
        
        frm = qv[qv.frame == i]
        
        y_vals = frm.biny.unique()
     
        for y in enumerate(y_vals):
            nxvel = frm.groupby('binx', as_index=False)['nxvel'].mean() 
            qv.update(nxvel)
            nyvel = frm.groupby('binx', as_index=False)['nyvel'].mean()
            qv.update(nyvel)
            hyp_vel = frm.groupby('binx', as_index=False)['hyp_vel'].mean()
            qv.update(hyp_vel)
            biny = frm.groupby('binx', as_index=False)['biny'].mean()
            qv.update(biny)
            
    print('\n')
    print('Final output data: ')
    print('\n')
    print(qv.head())
    print('\n')
    
    sys.stdout.write("\r Averaging across (x, y) co-ordinates: {:.2%}".format(1/1))  
    qv.to_csv(general_directory + "/output_data.csv") # Export of qv dataframe to confirm correct formatting before plotting graphs
    
for c, param in enumerate(pkl_param):
    print('Performing calculations... ')
    print('\n')
    qv = pd.read_csv(data_dirs[c] + '/data_quiver.csv')
    quiver_df(qv, data_dirs[c])
    print('\n')
    print('All done. Data now ready for analysis in cells below! ')

In [ ]:
# Quiverkey vector plot

def quiver_plot(qv, general_directory):
    
    if not os.path.exists(general_directory + "/vector_frames/"):
        os.makedirs(general_directory + "/vector_frames/")
    frames = qv.frame.unique()
    
    def quiver_iter(i):
        quiv = qv[qv.frame == i]
        plot.set_offsets(np.column_stack([quiv.binx, quiv.biny]))        
        plot.set_UVC(quiv.nxvel, quiv.nyvel, quiv.hyp_vel)
        plt.title("t = " + str(i) + "s")
        #plt.colorbar(ax=ax)
        plt.savefig(general_directory + "/vector_frames/vector_" + str(i) + ".png")
        
    for i in frames: 
        sys.stdout.write("\r Producing vector plots: {:%}".format(i/len(frames)))
        sys.stdout.flush()
        
        fig = plt.figure(figsize=(5,13))
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlim(0, 500)
        ax.set_ylim(0, 1300)
        plot = ax.quiver([], [], [], [], cmap='RdBu', headlength=3)
        quiver_iter(i)
        
    
        sys.stdout.write("\r Producing vector plots: {:%}".format(1/1))  
        
    import imageio
    images = []
    for i in frames:
        images.append(imageio.imread(general_directory + "/vector_frames/vector_" + str(i) + ".png"))
    imageio.mimsave((general_directory + "/vector_frames/vector.gif"), images)
    
for c, param in enumerate(pkl_param):
    # the process_pickle_quiver function is contained within a separate file - process_pickle_quiver.py
    qv = pd.read_csv(data_dirs[c] + '/output_data.csv')
    print('Producing vector plot...')
    print('\n')
    quiver_plot(qv, data_dirs[c])    
    print('\n')
    print('Finished')  

In [ ]:
# Seaborn plot 
warnings.filterwarnings("ignore",category=RuntimeWarning)

def contourf_plot(qv, general_directory):
    
    if not os.path.exists(general_directory + "/heatmap_frames/"):
        os.makedirs(general_directory + "/heatmap_frames/")
    
    qv = qv.drop(columns=['nxvel', 'nyvel'])
    # create figure form matplotlib.pyplot
    frames = qv.frame.unique()
    snsfig = plt.figure() 
    
    def heatmap_iter(i):
        heatdf = qv[qv.frame == i]
        heatz = griddata((heatdf.binx, heatdf.biny), heatdf.hyp_vel.ravel(), (heatx[None, :], heaty[:, None]), method='nearest')
        im = plt.contourf(heatx, heaty, heatz, cmap='RdBu', vmax=1, vmin=0)
        plt.colorbar(ax=axh)
        plt.title("t = " + str(i) + "s")
        plt.savefig(general_directory + "/heatmap_frames/heatmap_" + str(i) + ".png")
        
    for i in frames: 
        sys.stdout.write("\r Producing heatmap plots: {:%}".format(i/len(frames)))
        sys.stdout.flush()
        
        heatfig = plt.figure(figsize=(5,13))
        axh = heatfig.add_subplot(1, 1, 1)
        axh.set_xlim(0, 500)
        axh.set_ylim(0, 1300)
        heatx = np.linspace(0, 500, 11)
        heaty = np.linspace(0,1300, 27)
        heatmap_iter(i)
    
        sys.stdout.write("\r Producing heatmap plots: {:%}".format(1/1))  
        
    import imageio
    images = []
    for i in frames:
        images.append(imageio.imread(general_directory + "/heatmap_frames/heatmap_" + str(i) + ".png"))
    imageio.mimsave((general_directory + "/heatmap_frames/heatmap.gif"), images) 
        
for c, param in enumerate(pkl_param):
    # the process_pickle_quiver function is contained within a separate file - process_pickle_quiver.py
    qv = pd.read_csv(data_dirs[c] + '/output_data.csv')
    print('Heatmap data plots')
    print('\n')
    seaborn_plot(qv, data_dirs[c])    
    print('\n')
    print('Finished')  

In [ ]:
# Run calcs for all animation plots

for c, param in enumerate(pkl_param):
    # the process_pickle_quiver function is contained within a separate file - process_pickle_quiver.py
    qv = pd.read_csv(data_dirs[c] + '/output_data.csv')
    print('Producing vector plot...')
    print('\n')
    quiver_plot(qv, data_dirs[c])    
    print('\n')
    print('Producing heatmap plot...')
    print('\n')
    contourf_plot(df, data_dirs[c])    
    print('\n')
    print('Finished')  